# 8.2. Modelling with AutoGluon



In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('./dataset/train_ts_feature_added.csv')
df_train['datetime_iso'] = pd.to_datetime(df_train['datetime_iso'])
df_train['datetime_iso'] = df_train['datetime_iso'].dt.tz_localize(None)
df_train['item_id'] = 'T1'

df_test = pd.read_csv('./dataset/test_ts_feature_added.csv')
df_test['datetime_iso'] = pd.to_datetime(df_test['datetime_iso'])
df_test['datetime_iso'] = df_test['datetime_iso'].dt.tz_localize(None)
df_test_date = df_test['datetime_iso']
df_test['item_id'] = 'T1'

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341880 entries, 0 to 341879
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   datetime_iso        341880 non-null  datetime64[ns]
 1   temp                341880 non-null  float64       
 2   d_point             341880 non-null  float64       
 3   feels               341880 non-null  float64       
 4   min_temp            341880 non-null  float64       
 5   max_temp            341880 non-null  float64       
 6   prssr               341880 non-null  float64       
 7   hum                 341880 non-null  float64       
 8   wind_spd            341880 non-null  float64       
 9   wind_deg            341880 non-null  float64       
 10  rain_1h             341880 non-null  float64       
 11  clouds              341880 non-null  float64       
 12  day_of_the_week     341880 non-null  int64         
 13  month               341880 no

In [4]:
# Kita coba pake hourly_yearly_mean sama temp  aja
df_train = df_train[['datetime_iso','rain_1h','item_id','hourly_yearly_mean','temp','hum']]
df_test = df_test[['datetime_iso','rain_1h','item_id','hourly_yearly_mean','temp','hum']] 

## Modeling

In [5]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

/home/rayhanadi/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/rayhanadi/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [6]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column="item_id",
    timestamp_column="datetime_iso"
)
train_data.head()

rain_1h  hourly_yearly_mean   temp   hum
item_id timestamp                                                    
T1      1979-01-01 00:00:00     0.00            0.061282  24.75  95.0
        1979-01-01 01:00:00     0.00            0.042564  24.58  95.0
        1979-01-01 02:00:00     0.00            0.091538  26.60  86.0
        1979-01-01 03:00:00     0.13            0.257179  27.31  84.0
        1979-01-01 04:00:00     0.34            0.718974  27.41  87.0

In [7]:
df_train.columns

Index(['datetime_iso', 'rain_1h', 'item_id', 'hourly_yearly_mean', 'temp',
       'hum'],
      dtype='object')

In [8]:
df_test.describe()

,rain_1h,hourly_yearly_mean,temp,hum
count,0.0,49368.000000,49368.000000,49368.000000
mean,NaN,0.371508,27.297018,88.125020
std,NaN,0.366102,2.241756,9.195383
min,NaN,0.000000,22.450000,63.000000
25%,NaN,0.123846,25.510000,82.000000
50%,NaN,0.232308,26.710000,91.000000
75%,NaN,0.482821,29.000000,95.000000
max,NaN,2.270769,34.210000,100.000000


In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=49368,
    path="autogluon-m4-hourly",
    target="rain_1h",
    known_covariates_names=['temp','hum', 'hourly_yearly_mean'],
    eval_metric="RMSE",
)

predictor.fit(
    train_data,
    presets="best_quality",
    time_limit=1800,
    excluded_model_types=["RecursiveTabular","DeepAR"]
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: best_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': ['RecursiveTabular', 'DeepAR'],
 'hyperparameter_tune_kwargs': {'num_trials': 3,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 49368,
 'random_seed': None,
 'target': 'rain_1h',
 'time_limit': 1800,
 'verbosity': 2}
Provided training data set with 341880 rows, 1 items (item = single time series). Average time series length is 341880.0. Data frequency is 'H'.
AutoGluon will save models to autogluon-m4-hourly/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

